# Your API Key
Please mount your google drive before executing below code.

In [6]:
import json
# Load environment object from JSON
file_path = '/content/drive/MyDrive/keys/keys.json'  # Replace with the actual file path
with open(file_path, 'r') as file:
    loaded_object = json.load(file)

OPEN_AI_API_KEY = loaded_object['OPEN_AI_API_KEY']

#Environment Setup

*   Pip install all packages
*   Importing required libaries
*   Setup OpenAI API key
*   Initiatie LLM model



In [2]:
!pip install openai -q
!pip install langchain -q
!pip install chromadb -q
!pip install tiktoken -q
!pip install pypdf -q
!pip install unstructured[local-inference] -q
!pip install gradio -q

In [19]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain

import os
os.environ["OPENAI_API_KEY"] = OPEN_AI_API_KEY

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

In [4]:
import PIL
print(PIL.__version__)

9.5.0


In [5]:
!pip uninstall Pillow
!pip install --upgrade Pillow
print(PIL.__version__)

Found existing installation: Pillow 9.5.0
Uninstalling Pillow-9.5.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/PIL/*
    /usr/local/lib/python3.10/dist-packages/Pillow-9.5.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libXau-154567c4.so.6.0.0
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libbrotlicommon-92722cb2.so.1
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libbrotlidec-db4b3db6.so.1.0.9
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libfreetype-cb9caf6f.so.6.19.0
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libharfbuzz-3543f599.so.0.60710.0
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libjpeg-f2134fdd.so.62.3.0
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/liblcms2-12745711.so.2.0.15
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/liblzma-95592ee6.so.5.4.2
    /usr/local/lib/python3.10/dist-packages/Pillow.libs/libopenjp2-78c47f58.so.2.5.0
    /usr/local/lib/python3.10/d

# Load data from your location
In my example my files are located on my google drive.

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:

from langchain.document_loaders import DirectoryLoader

pdf_loader = DirectoryLoader('/content/drive/MyDrive/Colab Notebooks/test/', glob="**/*.txt")


loaders = [pdf_loader]
documents = []
for loader in loaders:
  documents.extend(loader.load())
print (f"Total number of documents: {len(documents)}")

Total number of documents: 4


# File data processing
Split text by defined chunk size, embedding and store the data in vector store.

In [69]:
from langchain import vectorstores
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
vectorstores = Chroma.from_documents(documents, embeddings)


# Constructe the LLM model 
Constructe the LLM model with the created vector store.

In [70]:
qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0), vectorstores.as_retriever())

# Start chatting!

In [10]:
chat_history = []


In [45]:
query = "In all the RBA statments you stored here, how many times RBA decided to lift interest rates, please give me some example dates and the interest rate decision outcome"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

'The RBA decided to lift interest rates in two of the statements:\n\n1. Media Release Statement by Glenn Stevens, Governor: Monetary Policy Decision Number 2010-04 Date 2 March 2010 - The Board decided to raise the cash rate by 25 basis points to 4.0 per cent, effective 3 March 2010.\n\n2. Media Release Statement by Glenn Stevens, Governor: Monetary Policy Decision Number 2010-06 Date 6 April 2010 - The Board decided to raise the cash rate by 25 basis points to 4.25 per cent, effective 7 April 2010.'

In [46]:
query = "Could you summarise the main points the April 2021 RBA statement is trying to deliver"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

'The Reserve Bank of Australia (RBA) has decided to maintain the current policy settings, including the targets of 10 basis points for the cash rate and the yield on the 3-year Australian Government bond, as well as the parameters of the Term Funding Facility and the government bond purchase program. The global economy is recovering, although the recovery is uneven, and inflation remains low. The Australian economy is recovering faster than expected, with GDP increasing by 3.1% in the December quarter, boosted by a further lift in household consumption as the health situation improved. The recovery is expected to continue, with above-trend growth this year and next. Wage and price pressures are subdued and are expected to remain so for some years. The Board is committed to maintaining highly supportive monetary conditions until its goals of full employment and inflation consistent with the target are achieved. The Board will not increase the cash rate until actual inflation is sustaina

In [47]:
query = "In the April 2021 RBA statement, does the sentiment sounds like RBA is going to lift the interest rate in the next meeting?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

'No, the sentiment in the April 2021 RBA statement does not suggest that the RBA is going to lift the interest rate in the next meeting. The statement mentions that the Board decided to maintain the cash rate target at 10 basis points and the interest rate on Exchange Settlement balances at zero per cent. The statement also mentions that the Board is committed to achieving the goals of full employment and inflation consistent with the target, and that the current measures will provide the continuing monetary support that the economy needs as it transitions from the recovery phase to the expansion phase.'

In [71]:
query = "What are the key points delivered in the 2 May 2023 RBA statement?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

"The key points delivered in the 2 May 2023 RBA statement are:\n\n- The Board decided to increase the cash rate target by 25 basis points to 3.85 per cent and the rate paid on Exchange Settlement balances by 25 basis points to 3.75 per cent.\n- Inflation in Australia has passed its peak, but at 7 per cent is still too high and it will be some time yet before it is back in the target range.\n- Goods price inflation is clearly slowing due to a better balance of supply and demand following the resolution of the pandemic disruptions. But services price inflation is still very high and broadly based and the experience overseas points to upside risks.\n- The labour market remains very tight, with the unemployment rate at a near 50-year low. Many firms continue to experience difficulty hiring workers, although there has been some easing in labour shortages and the number of vacancies has declined a little.\n- The Board's priority remains to return inflation to target. High inflation makes lif

In [72]:
query = "In the May 2023 RBA statement, does the sentiment sound like RBA is going to lift the interest rate in the next meeting?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

'The May 2023 RBA statement suggests that the Board has already increased the cash rate target by 25 basis points to 3.85 per cent and the rate paid on Exchange Settlement balances by 25 basis points to 3.75 per cent. The statement also indicates that further tightening of monetary policy may be required to ensure that inflation returns to target in a reasonable timeframe, but that will depend upon how the economy and inflation evolve. Therefore, it is possible that the RBA may consider lifting the interest rate in the next meeting if the economic and inflation outlook warrants it.'